# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split

# Read the Clean CSV

In [3]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1_Clean.csv')
df = pd.read_csv(file_path)
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [4]:
# Create our features
df_encoded = pd.get_dummies(df)
X = df_encoded.drop(["loan_status_low_risk", "loan_status_high_risk"], axis=1)

# Create our target
y = df[["loan_status"]]

In [5]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [6]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [7]:
# Train - Test - Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [8]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(51612, 95)
(51612, 1)
(17205, 95)
(17205, 1)


# Oversampling

Compare two oversampling algorithms to determine which algorithm results in the best performance. Oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [9]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'loan_status': 1})

In [10]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [11]:
# Generate predictions
y_pred = model.predict(X_test)
pred_act_df = pd.DataFrame({'Prediction': y_pred, 'Actual': y_test.loan_status})
pred_act_df = pred_act_df.reset_index(drop=True)
pred_act_df.head()

,Prediction,Actual
0,high_risk,low_risk
1,high_risk,low_risk
2,high_risk,low_risk
3,high_risk,low_risk
4,high_risk,low_risk


In [12]:
# Calculate percentage of correct predictions (accuracy score)
pred_act_df['Accurate'] = (pred_act_df.Prediction == pred_act_df.Actual)
print(len(pred_act_df[pred_act_df.Accurate==True])/len(pred_act_df.Accurate))

# Check
print(accuracy_score(y_test, y_pred))

0.6997965707643127
0.6997965707643127


In [13]:
# Calculate True Positive (TP) Count assuming 'True' is Low-Risk
## Where Actual = Predicted = low_risk
len(pred_act_df[(pred_act_df.Prediction == 'low_risk') & (pred_act_df.Actual == 'low_risk')])

11988

In [14]:
# Calculated the balanced accuracy score
ba_score = balanced_accuracy_score(y_test, y_pred)
ba_score

0.6490083034192683

In [15]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, columns=['Predicted False (High-Risk)', 'Predicted True (Low-Risk)'],
                    index=['Actual False (High-Risk)', 'Actual True (Low-Risk)'])
cm_df

,Predicted False (High-Risk),Predicted True (Low-Risk)
Actual False (High-Risk),52,35
Actual True (Low-Risk),5130,11988


In [16]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.60      0.70      0.02      0.65      0.41        87
   low_risk       1.00      0.70      0.60      0.82      0.65      0.42     17118

avg / total       0.99      0.70      0.60      0.82      0.65      0.42     17205



### SMOTE Oversampling

In [17]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled2, y_resampled2 = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)
Counter(y_resampled2)

Counter({'loan_status': 1})

In [18]:
# Train the Logistic Regression model using the resampled data
model2 = LogisticRegression(solver='lbfgs', random_state=1)
model2.fit(X_resampled2, y_resampled2)

LogisticRegression(random_state=1)

In [19]:
# Generate predictions
y_pred2 = model2.predict(X_test)
pred_act_df2 = pd.DataFrame({'Prediction': y_pred2, 'Actual': y_test.loan_status})
pred_act_df2 = pred_act_df2.reset_index(drop=True)
pred_act_df2.head()

,Prediction,Actual
0,high_risk,low_risk
1,high_risk,low_risk
2,low_risk,low_risk
3,high_risk,low_risk
4,high_risk,low_risk


In [20]:
# Calculated the balanced accuracy score
ba_score2 = balanced_accuracy_score(y_test, y_pred2)
ba_score2

0.6259120264613575

In [21]:
# Display the confusion matrix
cm2 = confusion_matrix(y_test, y_pred2)
cm2_df = pd.DataFrame(cm2, columns=['Predicted False (High-Risk)', 'Predicted True (Low-Risk)'],
                    index=['Actual False (High-Risk)', 'Actual True (Low-Risk)'])
cm2_df

,Predicted False (High-Risk),Predicted True (Low-Risk)
Actual False (High-Risk),55,32
Actual True (Low-Risk),6511,10607


In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred2))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.63      0.62      0.02      0.63      0.39        87
   low_risk       1.00      0.62      0.63      0.76      0.63      0.39     17118

avg / total       0.99      0.62      0.63      0.76      0.63      0.39     17205



# Undersampling

Test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. Undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [23]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled3, y_resampled3 = cc.fit_resample(X_train, y_train)
Counter(y_resampled3)

Counter({'loan_status': 1})

In [24]:
# Train the Logistic Regression model using the resampled data
model3 = LogisticRegression(solver='lbfgs', random_state=1)
model3.fit(X_resampled3, y_resampled3)

LogisticRegression(random_state=1)

In [25]:
# Generate predictions
y_pred3 = model3.predict(X_test)
pred_act_df3 = pd.DataFrame({'Prediction': y_pred3, 'Actual': y_test.loan_status})
pred_act_df3 = pred_act_df3.reset_index(drop=True)
pred_act_df3.head()

,Prediction,Actual
0,high_risk,low_risk
1,high_risk,low_risk
2,high_risk,low_risk
3,high_risk,low_risk
4,low_risk,low_risk


In [26]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred3)

0.5290690178920354

In [27]:
# Display the confusion matrix
cm3 = confusion_matrix(y_test, y_pred3)
cm3_df = pd.DataFrame(cm3, columns=['Predicted False (High-Risk)', 'Predicted True (Low-Risk)'],
                    index=['Actual False (High-Risk)', 'Actual True (Low-Risk)'])
cm3_df

,Predicted False (High-Risk),Predicted True (Low-Risk)
Actual False (High-Risk),53,34
Actual True (Low-Risk),9433,7685


In [28]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred3))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.61      0.45      0.01      0.52      0.28        87
   low_risk       1.00      0.45      0.61      0.62      0.52      0.27     17118

avg / total       0.99      0.45      0.61      0.62      0.52      0.27     17205



# Combination (Over and Under) Sampling

Test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. Resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [29]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled4, y_resampled4 = smote_enn.fit_resample(X, y)
Counter(y_resampled4)

Counter({'loan_status': 1})

In [30]:
# Train the Logistic Regression model using the resampled data
model4 = LogisticRegression(solver='lbfgs', random_state=1)
model4.fit(X_resampled4, y_resampled4)

LogisticRegression(random_state=1)

In [31]:
# Calculated the balanced accuracy score
y_pred4 = model4.predict(X_test)
balanced_accuracy_score(y_test, y_pred4)

0.6375241226214794

In [32]:
# Display the confusion matrix
cm4 = confusion_matrix(y_test, y_pred4)
cm4_df = pd.DataFrame(cm4, columns=['Predicted False (High-Risk)', 'Predicted True (Low-Risk)'],
                    index=['Actual False (High-Risk)', 'Actual True (Low-Risk)'])
cm4_df

,Predicted False (High-Risk),Predicted True (Low-Risk)
Actual False (High-Risk),61,26
Actual True (Low-Risk),7294,9824


In [33]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.70      0.57      0.02      0.63      0.41        87
   low_risk       1.00      0.57      0.70      0.73      0.63      0.40     17118

avg / total       0.99      0.57      0.70      0.72      0.63      0.40     17205

